<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/4/47/Acronimo_y_nombre_uc3m.png"/>

<img src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" width=15%/>
</center>   






# Un breve tour por BERT

BERT, modelo auto-encoding, es probablemente uno de los transformers más conocidos. 

En su pre-entrenamiento se utilizaron dos estrategias: predicción de tokens enmascarados y predecir si dos oraciones son consecutivas. Es un modelo bidireccional. BERT fue pre-entrenado con una gran colección de textos (BookCorpus y Wikipedia). 

BERT puede ser facilmente ajustado para cualquier tarea de NLP, añadiendo simplemente un capa de salida adecuada para la tarea. En su momento, BERT fue capaz de superar los resultados de los sistemas desarrollados hasta el momento en muchas tareas de NLP como la clasificación de textos o el reconocimiento de entidades. Sin embargo, aunque BERT fue entrenado utilizando una tarea de modelado de lenguaje al predecir tokens enmascarados, BERT no es un modelo apropiado para tareas de generación de texto. 

El siguiente [paper](#https://aclanthology.org/N19-1423/) describe en detalle la arquitectura de BERT.


En los siguientes notebooks, aprenderemos a ajustar BERT para distintas tareas de NLP. HuggingFace proporciona algunas clases pre-definidas que pueden er utilizados para tareas específicas. 

El primer paso será instalar la librería transformers de HuggingFace.

In [ ]:
!pip install transformers

## Reviewing BERT parameters

We can see some properties:
- *max_position_embeddings*:512, tamaño de la secuencia más larga que BERT puede procesar. 
- *hidden_size*: 768. Dimensión de los vectores. 
- *vocab_size*: el vocabulario de BERT tiene un total de 30,522 tokens únicos.
- *num_attention_heads*: 12, es el número de cabezas de atención en cada capa de atención.
- *num_hidden_layers*: 12, número de capas ocultas.
- *pad_token_id*: 0, es el identificador para el token de padding. 
- *position_embedding_type*: en BERT, se utiliza una posición absoluta para representar los tokens. .

Puedes encontrar más información en el siguiente [link](#https://huggingface.co/docs/transformers/model_doc/bert).

In [ ]:
from transformers import BertModel, BertConfig

# Initializing a BERT bert-base-uncased style configuration
configuration = BertConfig()

# Initializing a model from the bert-base-uncased style configuration
model = BertModel(configuration)

# Accessing the model configuration
configuration = model.config

print(configuration)

## BertForNextSentencePrediction
Como se ha dicho antes, predecir si dos oraciones eran consecutivas fue una de las estrategias utilizadas para pre-entrenar BERT. 
La librería transformers nos proporciona una clase que nos permite utilizar  esta estrategia sobre pares de oraciones. La clase recibe como entrada dos oraciones, y devuelve la probabilidad de que esas dos oraciones puedan ocurrir juntas. 
Antes de utilizar la clase, debemos cargar el tokenizador asociado con el modelo BERT. Esta tokenizador nos permite representar los textos en el formato adecuado que necesita BERT (tokens_ids, token_type_ids, attention_mask).


In [ ]:
from transformers import BertTokenizer, BertForNextSentencePrediction

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForNextSentencePrediction.from_pretrained("bert-base-uncased")

first_sentence = "Leonardo da Vinci was an Italian man who lived in the time of the Renaissance."
second_sentence = "He is famous for his paintings."

# the tokenizer transform the two sentences to the input format that BERT needs. 
encoding = tokenizer(first_sentence, second_sentence, return_tensors="pt")

print(encoding)

Como ya explicamos en notebooks anteriores, el tokenizador devuelve un diccionario con tres campos: input_ids, token_type_ids, attention_mask. 

In [ ]:
print(encoding.input_ids)       #ids of the tokens
print(encoding.token_type_ids) #0's for tokens of the first sentence, 1's for tokens of the second one
print(encoding.attention_mask)  #1's for tokens, 0's for padding. 


Ahora podemos utilizar la clase **BertForNextSentencePrediction** para calcular la probabilidad.
Aplicamos el modelo sobre el encoding de las dos oraciones. 

El modelo devuelve un objeto vector (Tensor), **logists**, cuyo primer elemento es la probabilidad de que las dos oraciones pueden ocurrir juntas. Podemos ver que obtenemos un valor alto alto que indica que las dos oraciones pueden ocurrir juntas.


In [ ]:
outputs = model(**encoding) 
logits = outputs.logits
print(type(logits))
print(logits)

# assert logits[0, 0] < logits[0, 1]  # next sentence was random

Sin embargo, en el siguiente ejemplo, la probabilidad es negativa, lo que indica que ambas oraciones no es probable que aparezcan juntas. 

In [ ]:
first_sentence = "Leonardo da Vinci was an Italian man who lived in the time of the Renaissance."
second_sentence = "Pizza is an Italian food that was created in Italy."

# the tokenizer transform the two sentences to the input format that BERT needs. 
encoding = tokenizer(first_sentence, second_sentence, return_tensors="pt")

outputs = model(**encoding)
logits = outputs.logits
print(logits)
# assert logits[0, 0] < logits[0, 1]  # next sentence was random

## TFBertForMaskedLM

Vamos a probar también la otra estrategia utilizada para pre-entrenar BERT: predecir un token enmascarado. 
La librería **transformers** ya incluye una clase, TFBertForMaskedLM, que implementa la tarea. En este caso, la entrada será una oración donde un token ha sido reemplazado por una máscara [𝑀𝐴𝑆𝐾]. 

In [ ]:
from transformers import TFBertForMaskedLM
import tensorflow as tf


model = TFBertForMaskedLM.from_pretrained("bert-base-uncased")
encoding = tokenizer("This [MASK] is beautiful.", return_tensors="tf")

print(encoding)

In [ ]:
output = model(**encoding)
logits = output.logits
print(logits)


Después de aplicar el modelo sobre la entrada codificada, este devuelve un tensor  con dimensión (1,7,30522).

- donde 7 es el número de tokens en la oración de entrada: "$[CLS]$ This $[MASK]$ is beautiful. $[SEP]$". Tiene 7 tokens al incluir los tokens especiales $[CLS]$ y $[SEP]$.
- 30.522 es el tamaño de vocabulario de BERT.




La salida está en **logits[0]**, es un tensor (vector) de forma 7 x 30522, cada fila representa un token y cada columna representa una de las 30.522 tokens del vocabulario de BERT. BERT ha calculado la probabilidad del token para cada uno de los tokens del vocabulario, así en la fila i y columna j, el tensor contiene la probabilidad de que el token representado en la fila i sea el token j del vocabulario. 
Por ejemplo, $logits[0][2]$ va a contener el vector de probabilidades que el modelo ha calculado para el token $[MASK]$ en la oración "$[CLS]$ This $[MASK]$ is beautiful.$[SEP]$". La probabilidad más alta se corresponderá con el token predecido para $[MASK]$. 

En primer lugar, vamos a calcular de forma automática el índice de $[MASK]$ en el vocabulario y en la oración de entrada. 


In [ ]:
print(tokenizer.mask_token_id)
# check that the decoding of this id is MASK
print(tokenizer.decode(tokenizer.mask_token_id))

Por tanto, $[MASK]$ tiene el índice 103 en el vocabulario de BERT. 
Ahora vamos a obtener el índice de $[MASK]$ en la oración de entrada: 

In [ ]:
print(encoding.input_ids)
print(type(encoding.input_ids))
print(type(encoding.input_ids.numpy()))

# convert from Tensor to list: tensor -> numpy -> list
input_sentence_list = (encoding.input_ids.numpy().tolist())
print(input_sentence_list)
input_sentence_list = input_sentence_list[0]
print(input_sentence_list)


¿Cuál es la posición del token mask_token_id en la oración? 

In [ ]:
# find the index 
mask_token_index = input_sentence_list.index(tokenizer.mask_token_id)
mask_token_index
print("posición de [MASK] en la oración:", mask_token_index)


Ahora vamos a obtener el tensor que ha calculado BERT para ese token:

In [ ]:
output = logits[0]
print(print(type(output)))
# from Tensor to numpy to list
output_list = output.numpy().tolist()
predicted_list_mask = output_list[mask_token_index]
print("predictions for mask:", output_list[mask_token_index])



Calculamos el índice cuyo valore es el mayor del tesnro. Para eso usamos la función **argmax**:

In [ ]:
predicted_id_token_mask = tf.math.argmax(predicted_list_mask, axis=-1)
predicted_id_token_mask

Para conocer el token que ha inferido el modelo, necesitamos utilizar la función **decode**: 


In [ ]:
tokenizer.decode(predicted_id_token_mask)

Hemos recuperado el token, transformando el tensor a una lista y operando con sus funciones. Sin embargo, en lugar de hacer esto, es recomendable utilizar la operaciones para trabajar con Tensor (un tensor es una matriz multidimensional). 
Ver: https://www.tensorflow.org/guide/tensor

Por ejemplo, for obtener el índice de $[MASK]$ en la oración, debemos hacer lo siguiente: 

In [ ]:
mask_token_index = tf.where((encoding.input_ids == tokenizer.mask_token_id)[0])
print(mask_token_index)

In [ ]:
selected_logits = tf.gather_nd(logits[0], indices=mask_token_index)

# we have to take the maximum value
predicted_id_token_mask = tf.math.argmax(predicted_list_mask, axis=-1)
#finally we decode to obtain the word
tokenizer.decode(predicted_id_token_mask)

### Ejercicio: 
Prueba tu mismo/a con otras oraciones: "Please, read this $[MASK]$."

In [ ]:
inputs = tokenizer("Please, read this [MASK].", return_tensors="tf")
logits = model(**inputs).logits

mask_token_index = tf.where((inputs.input_ids == tokenizer.mask_token_id)[0])
print(mask_token_index)
selected_logits = tf.gather_nd(logits[0], indices=mask_token_index)

# we have to take the maximum value
predicted_id_token_mask = tf.math.argmax(selected_logits, axis=-1)
#finally we decode to obtain the word
tokenizer.decode(predicted_id_token_mask)

## BertForTokenClassification

La librería **transformes** proporciona también una clase que permite utilizar el modelo BERT para la tarea de reconocimiento de entidades (NER). NER se puede ver como una tarea donde el objetivo es clasificar cada token en una oración, y las clases siguen el estándar IOB (O no entidad, I- token interno de entidad). 

Vamos a cargar un modelo de BERT que ha sido ajustaedo a la tarea de NER. 

In [ ]:
from transformers import BertTokenizer, BertForTokenClassification
import torch

tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
model = BertForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")

inputs = tokenizer(
    "HuggingFace is a company based in Paris and New York", add_special_tokens=False, return_tensors="pt"
)

# we use the model to infer, so we need to disable gradient calculation
with torch.no_grad():
    logits = model(**inputs).logits
# logits

In [ ]:
predicted_token_class_ids = logits.argmax(-1)
print(predicted_token_class_ids)
# Note that tokens are classified rather then input words which means that
# there might be more predicted token classes than words.
# Multiple token classes might account for the same word
predicted_tokens_classes = [model.config.id2label[t.item()] for t in predicted_token_class_ids[0]]
predicted_tokens_classes
# ['O', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'I-LOC', 'O', 'I-LOC', 'I-LOC'] 


BERT necesita codificar la posición de cada token. Esta posición es necesaria para representar cómo un token en un posición afecta a otro token en una posición diferente. **BERT utiliza la posición absoluta de cada token** (desde 1 hasta la longitud máxima de la secuencia). Por este motivo, **el padding en BERT debe aplicarse a la derecha**.

En los siguientes notebooks, aprenderemos cómo ajustar BERT para la tarea de clasificación de textos. También estudiaremos otros modelos basados en BERT:  Distilbert, RoBERTa, etc. 

